# 전체 학습에 사용되었던 추가 데이터셋 생성
- 학습에는 데이콘에서 공유해주신 코드를 바탕으로 추가 데이터셋을 생성하였습니다
- 또한, 빠른 학습을 위하여 tfrecord 형태로 변환하여 사용하였습니다
- 컴페티션 기본 데이터는 data/ 하위 폴더에 있다고 가정합니다. (train.csv, sample_submission.csv, etc)
- 또한 train.zip, test.zip 역시 data/ 하위에 압축을 풀어놓았다고 가정하고 시작하겠습니다..

In [ ]:
# from IPython.display import clear_output

# !add-apt-repository -y ppa:alessandro-strada/ppa; 
# !apt-get update;
# !apt-get install -y google-drive-ocamlfuse;
# clear_output()

# !mkdir google_drive;
# !google-drive-ocamlfuse -headless -label dacon_smiles -id 406775554485-vqr231cgnpofc9mkm7sr0e3uq32emf11.apps.googleusercontent.com -secret iFy1t7pKRjOzBuWHbUB-cM8V;

# !sed -i 's/team_drive_id=0AOLSYhuNgxEsUk9PVA/team_drive_id=/' ~/.gdfuse/dacon_smiles/config
# !sed -i 's/team_drive_id=/team_drive_id=0AOLSYhuNgxEsUk9PVA/' ~/.gdfuse/dacon_smiles/config
# !google-drive-ocamlfuse -label dacon_smiles google_drive/

# # !fusermount -u google_drive

# # rdkit 2020.03.3 버전 다운로드
# !pip install kora -q
# import kora.install.rdkit

# import os
# import os.path as pth

# ### 저는 코랩에서 구글 드라이브를 네트워크 마운트해서 사용했기 때문에 경로가 이와 같이 됩니다.
# google_drive_base_path = 'google_drive/chemical/'

# clear_output()

In [ ]:
import os
import os.path as pth

data_base_path = 'data'
os.makedirs(data_base_path, exist_ok=True)

In [ ]:
# google_drive_base_path = 'google_drive/chemical/'

CID_dataset_dir = 'CID-SMILES'

In [ ]:
# if not pth.exists(pth.join(data_base_path, CID_dataset_dir)):
#     os.system('cp -r {}/data/{} data'.format(google_drive_base_path, CID_dataset_dir))

In [ ]:
import tensorflow as tf

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import functools

import random
import numpy as np
import pandas as pd
import os
import time
import cv2
from tqdm import tqdm
from glob import glob

import kora.install.rdkit

import rdkit
from rdkit import Chem
from rdkit import DataStructs
from rdkit import RDLogger
from rdkit.Chem import Draw
import multiprocessing

RDLogger.DisableLog('rdApp.*')  

from IPython.display import clear_output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


## 1. CID-SMILES에서 새로운 train set 추출하기

먼저 Pubchem에서 다운 받은 CID-SMILES 파일을 불러온 후 1억 개의 SMILES 분자식을 text라는 변수에 하나의 스트링으로 저장합니다.  
이후에 진행되는 프로세스는 데이콘에서 공유해주신 것과 동일합니다.



In [ ]:
# CID-SMILES 파일 경로
path = pth.join(data_base_path, CID_dataset_dir, 'CID-SMILES')

In [ ]:
f = open(path)

text = ""
for _ in tqdm(range(100000000)):
    s = f.readline()
    s = s.split('\t')[1]
    s = s.split('\n')[0]
    text+=s

chars = list(set(text))

In [ ]:
char_count = {}
for char in tqdm(chars):
    char_count[char] = text.count(char)
char_count

In [ ]:
pass_chars = []
for char,_ in sorted(char_count.items(), key=lambda x:x[1], reverse=False)[:len(char_count)//2]:
    pass_chars.append(char)

In [ ]:
del(text)

1.1. 더 많은 데이터를 학습하기 위해, 기존 CID-SMILES 데이터셋에서 새로운 데이터 100만개를 새로 생성합니다.


In [ ]:
f = open(path)

data = ""
for _ in tqdm(range(100000000)):
    s = f.readline()
    data+=s

data = data.split('\n')
print(len(data))
random.shuffle(data)

In [ ]:
max_length = 70
sample_n = 1000000

In [ ]:
### Original code
smiles_dict = {'SMILES':[]}
for i, d in tqdm(enumerate(data)):
    smiles = d.split('\t')[1]
    m = Chem.MolFromSmiles(smiles)
    if m != None and len(smiles) <= max_length:
        confirm = 1
        for pass_char in pass_chars:
            if smiles.find(pass_char) != -1:
                confirm = 0
        if confirm == 1:
            smiles_dict['SMILES'].append(smiles)
    if len(smiles_dict['SMILES']) == sample_n:
        break
        
len(smiles_dict['SMILES'])

In [ ]:
train_another = pd.DataFrame(smiles_dict)
train_another['file_name'] = 'train_another_' + train_another.index.astype('str') + '.png'
train_another = train_another[['file_name', 'SMILES']]
train_another.head(5)

In [ ]:
train_another['SMILES'].str.len().hist(bins=20, rwidth=0.8)

In [ ]:
# train_another.to_csv(pth.join(google_drive_base_path, 'data', 'train_another.csv'), index=False)
train_another.to_csv(pth.join(data_base_path, 'train_another.csv'), index=False)

1.2. 모델이 더 긴 데이터도 학습할 수 있게 길이가 70~98인 데이터도 30만개도 추가로 생성합니다.  
(양 끝에 '<', '>'가 추가되는 것을 고려)

In [ ]:
max_length = 98
sample_n = 300000

In [ ]:
smiles_dict = {'SMILES':[]}
for i, d in tqdm(enumerate(data)):
    smiles = d.split('\t')[1]
    m = Chem.MolFromSmiles(smiles)
    if m != None and len(smiles) <= max_length and len(smiles) > 70:
        confirm = 1
        for pass_char in pass_chars:
            if smiles.find(pass_char) != -1:
                confirm = 0
        if confirm == 1:
            smiles_dict['SMILES'].append(smiles)
    if len(smiles_dict['SMILES']) == sample_n:
        break
        
len(smiles_dict['SMILES'])

In [ ]:
train_over70 = pd.DataFrame(smiles_dict)
train_over70['file_name'] = 'train_over70_' + train_over70.index.astype('str') + '.png'
train_over70 = train_over70[['file_name', 'SMILES']]
train_over70.head(5)

In [ ]:
train_over70['SMILES'].str.len().hist(bins=20, rwidth=0.8)

In [ ]:
# train_over70.to_csv(pth.join(google_drive_base_path, 'data', 'train_over70.csv'), index=False)
train_over70.to_csv(pth.join(data_base_path, 'train_over70.csv'), index=False)

1.3. 모델이 더 다양한 데이터도 학습할 수 있게 빈도에 관계없이 길이가 ~98인 데이터도 30만개도 추가로 생성합니다.  

In [ ]:
max_length = 98
sample_n = 300000

In [ ]:
smiles_dict = {'SMILES':[]}
for i, d in tqdm(enumerate(data), position=0, leave=True):
    if len(d) == 0:
      continue
    smiles = d.split('\t')[1]
    m = Chem.MolFromSmiles(smiles)
    if m != None and len(smiles) <= max_length:
        for pass_char in pass_chars:
            if pass_char in smiles:
                smiles_dict['SMILES'].append(smiles)
                break
    if len(smiles_dict['SMILES']) == sample_n:
        break
        
len(smiles_dict['SMILES'])

In [ ]:
train_under_50per = pd.DataFrame(smiles_dict)
train_under_50per['file_name'] = 'train_under_50per_' + train_under_50per.index.astype('str') + '.png'
train_under_50per = train_under_50per[['file_name', 'SMILES']]
train_under_50per.head(5)

In [ ]:
train_under_50per['SMILES'].str.len().hist(bins=20, rwidth=0.8)

In [ ]:
# train_under_50per.to_csv(pth.join(google_drive_base_path, 'data', 'train_under_50per.csv'), index=False
train_under_50per.to_csv(pth.join(data_base_path, 'train_under_50per.csv'), index=False)

tfrecord로 만들기 전에, 먼저 추출한 smiles 식들을 이미지로 저장합니다.

In [ ]:
name_list = ['train_another', 'train_over70', 'train_under_50per']
for target_name in name_list:
    train = pd.read_csv(pth.join(data_base_path, target_name+'.csv'))
    print(train.shape)
    train_list = []
    for i in tqdm(range(train.shape[0]//10000+1)):
        train_list.append(np.array(train.loc[10000*i:10000*i+10000-1, ['SMILES', 'file_name']]))
    if not (os.path.isdir(target_name)):
        os.makedirs(os.path.join(target_name))
        
        
#     cpu_n = os.cpu_count()
    cpu_n = 16

    error_smiles = []
    def f(d):
        s, filename = d

        #예외 처리
        try:
            m = Chem.MolFromSmiles(s)
            img = Draw.MolToImage(m, size=(300,300))
            img = np.array(img)
            cv2.imwrite(pth.join(data_base_path, target_name, filename), img)
        except:
            error_smiles.append(s)

    for t in tqdm(train_list):
        if __name__ == '__main__':
            pool = multiprocessing.Pool(processes=cpu_n)
            pool.map(f,t)
            pool.close()
            pool.join()        
            
    for smiles in error_smiles:
        train = train[train['SMILES']!=smiles]
        
    #저장
    train.to_csv(pth.join(data_base_path, target_name+'.csv'), index=False)        

## 2. 추출한 csv와 생성한 이미지를 기반으로 tfrecord 생성

데이터를 읽는 오버헤드를 줄이기 위해 학습 데이터를 tfrecord형태로 새로 생성합니다

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import cv2

import matplotlib.pyplot as plt
from PIL import Image

from sklearn.model_selection import train_test_split, KFold, RepeatedKFold, GroupKFold
from sklearn.utils import shuffle

import numpy as np
import pandas as pd
import os
import os.path as pth
import shutil
import time
from tqdm import tqdm

import numpy as np
from PIL import Image

import rdkit
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit import DataStructs
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')  


from IPython.display import clear_output

from multiprocessing import Process, Queue
import datetime

In [ ]:
def calc_max_length(tensor):
    return max(len(t) for t in tensor)


def cvt_mol_caption(smiles_str):
    smiles_str = smiles_str[1:-1]
    chem_mol = Chem.MolFromSmiles(str(smiles_str))
    converted_smile_str = Chem.MolToSmiles(
        chem_mol,
        isomericSmiles=False, # Default: True,
        kekuleSmiles=False, # Default: False,
    #     rootedAtAtom=-1,
    #     canonical=False,
        allBondsExplicit=False,
        allHsExplicit=False
    )
#     print(smiles_str)
#     print(converted_smile_str)
#     print()
    converted_smile_str = '<'+converted_smile_str+'>'
    return converted_smile_str


def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _floatarray_feature(array):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=array))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def _validate_text(text):
    """If text is not str or unicode, then try to convert it to str."""
    if isinstance(text, str):
        return text
    elif isinstance(text, 'unicode'):
        return text.encode('utf8', 'ignore')
    else:
        return str(text)


def to_tfrecords(image_list, label_list, label_origin_list, tfrecords_name):
    print("Start converting")
    options = tf.io.TFRecordOptions(compression_type = 'GZIP')
    with tf.io.TFRecordWriter(path=pth.join(tfrecords_name+'.tfrecords'), options=options) as writer:
        for image_path, label, label_origin in tqdm(zip(image_list, label_list, label_origin_list), total=len(image_list), position=0, leave=True):

            filename = os.path.basename(image_path)
    #         image = np.array(Image.open(image_path))
            _binary_image = tf.io.read_file(image_path)
#             image = tf.image.decode_jpeg(_binary_image, channels=3)
    #         img = tf.dtypes.cast(img, tf.float32)
    #         label = Image.open(label_path)
    #         _binary_image = image.tobytes()
#             _binary_label = label.tobytes()
            label_100 = tf.keras.preprocessing.sequence.pad_sequences(
                np.array([label]), maxlen=100, padding='post'
            )

            each_real = ''.join([tokenizer.index_word.get(mol_i, '') for mol_i in label_100[0]])
            m_real = Chem.MolFromSmiles(each_real[1:-1])
            if m_real == None:
                continue

            string_set = tf.train.Example(features=tf.train.Features(feature={
    #             'height': _int64_feature(image.shape[0]),
    #             'width': _int64_feature(image.shape[1]),
                'image_raw': _bytes_feature(_binary_image),
                # 'label': _floatarray_feature(label),
                'label_100': _floatarray_feature(label_100[0]),
                'label_origin': _bytes_feature(label_origin.encode()),
    #             'std': _float_feature(image.std().astype(np.float32)),
                'filename': _bytes_feature(filename.encode()),
            }))

            writer.write(string_set.SerializeToString())    

Training 때 사용할 validation을 분리합니다. (Train:0.8, Validation:0.2)  
그냥 train_test_split을 사용하는 것보다는 5fold 형태로 분리하여 같은 데이터를 여러번 활용할 수도 있도록 하였습니다.  
tokenizer 같은 경우에는 가능한 모든 글자들을 대응시켜 향후에 어떤 데이터가 추가되어도 대응할 수 있도록 하였습니다.

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(lower=False, char_level=True)
all_token_list = [
    'c', 'C', '(', ')', '1', 'O', '=', '2', 'N', '<', '>', 'n', '[',
    ']', '3', '@', 'H', 'l', 'S', '-', 'F', '+', '4', 's', 'o', '#',
    'B', 'r', '.', '/', 'P', 'i', 'I', '5', '\\', 'e', 'A', 'a', 'g',
    '6', 'u', 't', 'T', 'M', 'b', 'K', 'Z', '8', 'd', '9', 'R', 'G',
    '7', 'L', 'V', 'h', 'W', 'p', 'm', 'E', 'Y', '0', 'U', 'f', 'D',
    'y', 'k', 'X', ' ', '^', '%', '$'
]
tokenizer.fit_on_texts(all_token_list)
top_k = len(tokenizer.word_index)

In [ ]:
name_list = ['train', 'train_another', 'train_over70', 'train_under_50per']
for train_path in name_list:
    with open(pth.join(data_base_path, train_path+'.csv'), 'r') as csv_file:
        data = csv_file.read()
        
    all_captions = []
    all_img_name_vector = []

    for line in data.split('\n')[1:-1]:
        image_id, smiles = line.split(',')
        caption = '<' + smiles + '>'
    #     caption = smiles
        full_image_path = pth.join(train_path, image_id)

        all_img_name_vector.append(full_image_path)
        all_captions.append(caption)

    train_captions, img_name_vector = shuffle(all_captions, all_img_name_vector, random_state=42)

    train_captions = np.array(train_captions)
    img_name_vector = np.array(img_name_vector)

    converted_train_captions = []
    for each_caption in tqdm(train_captions, position=0, leave=True):
        converted_train_captions.append(cvt_mol_caption(each_caption))
    train_captions = np.array(converted_train_captions)
    # train_captions = np.array(list(map(cvt_mol_caption, train_captions)))

    max_length = calc_max_length(train_captions)
    # max_length = 100

    train_seqs = tokenizer.texts_to_sequences(train_captions)
    cap_vector = tf.keras.preprocessing.sequence.pad_sequences(
        train_seqs, maxlen=max_length, padding='post'
    )
    cap_vector = cap_vector.astype(np.float32)

    keyword = train_path.replace('train_', '')

    temp_img_name_vector = img_name_vector
    temp_train_captions = cap_vector
    temp_origin_captions = train_captions

    rkf = RepeatedKFold(n_splits=5, n_repeats=1, random_state=7777)
    for rkf_i, (train_index, val_index) in enumerate(rkf.split(temp_img_name_vector)):
        fold_num = (rkf_i%5) + 1
        repeat_num = (rkf_i//5) + 1 
        
        if not fold_num <= 2:
            continue

        img_name_train, img_name_val = temp_img_name_vector[train_index], temp_img_name_vector[val_index]
        cap_train, cap_val = temp_train_captions[train_index], temp_train_captions[val_index]
        cap_origin_train, cap_origin_val = temp_origin_captions[train_index], temp_origin_captions[val_index]

        to_tfrecords(img_name_train, cap_train, cap_origin_train, pth.join(data_base_path, 'all_train_{}_fold_{:02d}'.format(keyword, fold_num)))
        to_tfrecords(img_name_val, cap_val, cap_origin_val, pth.join(data_base_path, 'all_val_{}_fold_{:02d}'.format(keyword, fold_num)))


Testset 또한 속도를 위해 tfrecord 형태로 변환해줍니다.

In [ ]:
TEST_PATH = pth.join(data_base_path, 'test')

with open(pth.join(data_base_path, 'sample_submission.csv'), 'r') as csv_file:
    data = csv_file.read()
    
test_img_path = []

for line in data.split('\n')[1:-1]:
    image_id, _ = line.split(',')
    full_image_path = pth.join(TEST_PATH, image_id)

    test_img_path.append(full_image_path)

In [ ]:
def to_tfrecords_test(image_list, label_list, tfrecords_base):
    print("Start converting")
    options = tf.io.TFRecordOptions(compression_type = 'GZIP')
    for image_path in tqdm(image_list, total=len(image_list), position=0, leave=True):
        tfrecord_filename = pth.join(tfrecords_base, pth.basename(image_path).split('.')[0])
        with tf.io.TFRecordWriter(path=pth.join(tfrecord_filename+'.tfrecords'), options=options) as writer:
            filename = os.path.basename(image_path)
            _binary_image = tf.io.read_file(pth.join('data', 'test', image_path))
            label_100 = tf.keras.preprocessing.sequence.pad_sequences(
                np.array([label]), maxlen=100, padding='post'
            )

            string_set = tf.train.Example(features=tf.train.Features(feature={
                'image_raw': _bytes_feature(_binary_image),
                # 'label_100': _floatarray_feature(label_100[0]),
                # 'label_origin': _bytes_feature(label_origin.encode()),
                'filename': _bytes_feature(filename.encode()),
            }))

            writer.write(string_set.SerializeToString())

In [ ]:
to_tfrecords_test(image_list=test_img_path, tfrecords_name=pth.join(data_base_path, 'test'))